
## Lunar Lander V3 - Deep Q-Learn


```
 pip install swig
 pip install gymnasium[box2d]
```
Imports necessários:


*   OS
*   Random
*   Numpy
*   Torch
*   Gymnasium









In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn # P/ Rede Neural
import torch.optim as optim # Otimizador
import torch.nn.functional as F # Funções de ativação, perda, convolução, etc (ReLU)
import torch.autograd as autograd # Diferenciação automática (gradiente)
from collections import deque

In [2]:
# Biblioteca Gymnasium
import gymnasium as gym
env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5, render_mode="human")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n # 0 - Do Nothing, 1 - Fire left, 2 - Fire down, 3 - Fire right
observation_space = env.observation_space
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)
print('Observation space: ', observation_space)


State shape:  (8,)
State size:  8
Number of actions:  4
Observation space:  Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)


In [3]:
# Parâmetros de aprendizado
learning_rate = 5e-4
batch_size =100 #
discount_factor = 0.99 # gamma
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

In [4]:
# Rede Neural
class Network(nn.Module):
  def __init__(self,state_size,number_actions,seed=42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64) # 3 Camadas de neurônios totalmente conectadas
    self.fc3 = nn.Linear(64, number_actions)


  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    q_values = self.fc3(x)
    return q_values

In [5]:
# Memória de aprendizado
class ReplayMemory(object):
    def __init__(self, capacity):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        # Deleta a memória mais antiga se estiver cheio (capacidade).
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        # Amostragem aleatória do tamanho do batch size
        experiences = random.sample(self.memory, k = batch_size)
        # Converte em tensores pytorch para serem processados pela rede neural
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        return (states, actions, rewards, next_states, dones)

In [6]:
class DQN():
  def __init__(self,state_size,number_actions, batch_size=100):
    self.batch_size = batch_size
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size =state_size
    self.action_size = number_actions
    self.memory = ReplayMemory(replay_buffer_size)
    self.model = Network(state_size,number_actions).to(self.device)
    self.target_model = Network(state_size,number_actions).to(self.device)
    self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
    self.t_step=0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > self.batch_size:
        experiences = self.memory.sample(self.batch_size)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.model.eval()
    with torch.no_grad():
      action_values = self.model(state)
    self.model.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = experiences
    next_q_targets = self.target_model(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.model(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.model, self.target_model, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [7]:
brain = DQN(state_size, number_actions)

In [8]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = brain.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    brain.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(brain.model.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -171.69
Episode 200	Average Score: -131.96
Episode 300	Average Score: -83.702
Episode 400	Average Score: -14.09
Episode 406	Average Score: -10.69

KeyboardInterrupt: 

In [41]:
import glob
import io
import base64
import imageio
import numpy as np # Certifique-se de que numpy está importado
from IPython.display import HTML, display

import gymnasium as gym

# Assumindo que 'brain' é uma instância da sua classe DQN devidamente inicializada e/ou treinada

def show_video_of_model(dqn, env_name):
    env = gym.make(env_name, render_mode='rgb_array') # Cria o ambiente com renderização em RGB (imagem por frame)

    state, _ = env.reset() # Reinicia o ambiente e recebe o estado inicial
    frames = [] # Lista que irá armazenar todos os frames do vídeo
    
    # Condição para o loop: rodar por um número fixo de passos ou até o episódio terminar e o ambiente resetar
    max_steps_per_episode = 1000 # Limite razoável de passos por episódio, ajuste conforme necessário
    current_step = 0
    
    # Loop principal para coletar frames. Roda até o episódio terminar (done) ou o limite de passos/frames ser atingido.
    # No caso de LunarLander, um episódio termina quando o lander pousa ou colide.
    while True:
        frame = env.render() # Captura o frame atual (imagem da simulação)
        
        # ESSENCIAL: VERIFICA SE O FRAME NÃO É NONE ANTES DE ADICIONAR E CONVERTER
        if frame is not None:
            frames.append(frame.astype(np.uint8)) # Adiciona o frame à lista, garantindo tipo uint8
        
        action = dqn.act(state, 0) # Usa o modelo DQN para escolher a ação (exploração = 0)
        
        # Aplica a ação e avança o ambiente no novo formato do Gymnasium
        state, reward, terminated, truncated, _ = env.step(action.item())
        
        # 'done' é verdadeiro se o episódio terminou por 'terminated' ou por 'truncated' (limite de tempo)
        done = terminated or truncated
        
        current_step += 1
        
        if done or current_step >= max_steps_per_episode: # Verifica se o episódio terminou ou o limite de passos foi atingido
            # Se o episódio terminou ou atingiu o limite, podemos sair do loop
            # Ou resetar o ambiente se quisermos continuar o vídeo por mais frames (como no seu exemplo anterior com max_frames)
            break # Sai do loop para finalizar o vídeo do episódio atual.
                  # Se quiser vídeo mais longo, reinicie o ambiente e continue o loop:
                  # state, _ = env.reset()
                  # current_step = 0 # Reinicia o contador de passos para o novo episódio
                  # if len(frames) >= 3000: break # Adicione uma condição para parar de coletar frames totais
                  
    env.close() # Fecha o ambiente após terminar
    imageio.mimsave('video.mp4', frames, fps=30) # Salva os frames como um vídeo .mp4 com 30 quadros por segundo

# Chamada da função. Certifique-se que 'brain' é sua instância do agente DQN.
# Ex: brain = DQN(state_size, number_actions)
# Se você tiver um modelo treinado salvo:
# brain.load() 
show_video_of_model(brain, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4') # Procura arquivos .mp4 no diretório
    if len(mp4list) > 0:
        mp4 = mp4list[0] # Seleciona o primeiro vídeo encontrado
        video = io.open(mp4, 'rb').read() # Lê o vídeo em modo binário
        encoded_video = base64.b64encode(video) # Codifica o vídeo em base64
        display(HTML(f'''
                <video width="640" height="480" controls>
                    <source src="data:video/mp4;base64,{encoded_video.decode('ascii')}" type="video/mp4">
                </video>
                ''')) # Mostra o vídeo embutido em HTML
    else:
        print("ERROR: Video not found") # Exibe mensagem de erro se nenhum vídeo for encontrado

show_video() # Executa a função para exibir o vídeo

TypeError: expected bytes, NoneType found